# Prelaod All data (CSV files)

In [15]:
import pandas as pd
from datetime import datetime

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Prepare all the data that will be used
_icd9_code = '32723' # (ADHD ICD-9 Code)
_admissions_csv = 'data/ADMISSIONS.csv.gz'
_diagnoses_csv = 'data/DIAGNOSES_ICD.csv.gz'
_patient_csv = 'data/PATIENTS.csv.gz'
_labevents = 'data/LABEVENTS.csv.gz'
_labitems = 'data/D_LABITEMS.csv.gz'

# Load Data
admissions_df = pd.read_csv(_admissions_csv, compression='gzip')
diagnoses_icd_df = pd.read_csv(_diagnoses_csv, compression='gzip')
patient_df = pd.read_csv(_patient_csv, compression='gzip')
labevents_df = pd.read_csv(_labevents, compression='gzip')
labitems_df = pd.read_csv(_labitems, compression='gzip')

# Clean column names (lower case & strip whitespace)
admissions_df.columns = admissions_df.columns.str.strip()
admissions_df.columns = admissions_df.columns.str.lower()
diagnoses_icd_df.columns = diagnoses_icd_df.columns.str.strip()
diagnoses_icd_df.columns = diagnoses_icd_df.columns.str.lower()
patient_df.columns = patient_df.columns.str.strip()
patient_df.columns = patient_df.columns.str.lower()
labevents_df.columns = labevents_df.columns.str.strip()
labevents_df.columns = labevents_df.columns.str.lower()
labitems_df.columns = labitems_df.columns.str.strip()
labitems_df.columns = labitems_df.columns.str.lower()

def calculate_age(dob, dod):
    try:
        dob = datetime.strptime(dob.split()[0], '%Y-%m-%d')
        dod = datetime.strptime(dod.split()[0], '%Y-%m-%d')
        return dod.year - dob.year
    except Exception as e:
        pass

# Prepare the subjects with apnea data (ICD-9 Code: 32723)

In [16]:
apnea_diagnoses = diagnoses_icd_df[diagnoses_icd_df["icd9_code"] == _icd9_code]
labs = pd.merge(labevents_df, labitems_df, on="itemid")

# Create new column for age & remove related columns
patient_df['dod_final'] = patient_df['dod'].fillna(patient_df['dod_hosp']).fillna(patient_df['dod_ssn'])
patient_df['age'] = patient_df.apply(lambda row: calculate_age(str(row['dob']), str(row['dod_final'])), axis=1)

# Filter out any subject whose age is negative or over 100
patient_age = patient_df[(patient_df['age'] > 0) & (patient_df['age'] < 100)]

# Combine admissions and remove unneeded columns
patient_admission = pd.merge(patient_age, admissions_df, on="subject_id")
patient_admission = patient_admission.drop(columns=['row_id_x', 'row_id_y', 'expire_flag', 'hadm_id', 'deathtime', 'admittime', 'dischtime', 'admission_location', 'discharge_location', 'hospital_expire_flag', 'edregtime', 'edouttime', 'has_chartevents_data', 'dod', 'dod_hosp', 'dod_ssn', 'dod_final'])

# Combine patients with labs & remove unneeded columns
patients_labs_all = pd.merge(patient_admission, labs, on="subject_id")
patients_labs_all = patients_labs_all.drop(columns=['dob', 'row_id_x', 'hadm_id', 'itemid', 'charttime', 'valuenum', 'row_id_y', 'loinc_code'])

# Clean up ethnicities
patients_labs_all['ethnicity'].replace(regex=r'^ASIAN\D*', value='ASIAN', inplace=True)
patients_labs_all['ethnicity'].replace(regex=r'^WHITE\D*', value='white', inplace=True)
patients_labs_all['ethnicity'].replace(regex=r'^HISPANIC\D*', value='HISPANIC', inplace=True)
patients_labs_all['ethnicity'].replace(regex=r'^BLACK\D*', value='BLACK', inplace=True)
patients_labs_all['ethnicity'].replace(["UNABLE TO OBTAIN", 'OTHER', 'PATIENT DECLINED TO ANSWER', 'UNKNOWN/NOT SPECIFIED'], value="Other/Unknown", inplace=True)
patients_labs_all['ethnicity'].loc[~patients_labs_all['ethnicity'].isin(patients_labs_all['ethnicity'].value_counts().nlargest(5).index.tolist())] = "Other/Unknown"
patients_labs_all['ethnicity'].value_counts()

# Drop any empty rows
patients_labs_cleaned = patients_labs_all.dropna()

# Filter out sleep apnea patients
patients_labs_apnea = pd.merge(patients_labs_cleaned, apnea_diagnoses, on="subject_id")
patients_labs_apnea = patients_labs_apnea.drop(columns=['row_id', 'hadm_id', 'seq_num', 'icd9_code'])

patients_labs_apnea[:10]

,subject_id,gender,age,admission_type,insurance,language,religion,marital_status,ethnicity,diagnosis,value,valueuom,flag,label,fluid,category
0,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,8.5,g/dL,abnormal,Hemoglobin,Blood,Hematology
1,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,10.4,%,abnormal,Lymphocytes,Blood,Hematology
2,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,29.9,%,abnormal,MCHC,Blood,Hematology
3,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,84.5,%,abnormal,Neutrophils,Blood,Hematology
4,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,17.0,%,abnormal,RDW,Blood,Hematology
5,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,2.96,m/uL,abnormal,Red Blood Cells,Blood,Hematology
6,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,38,mEq/L,abnormal,Bicarbonate,Blood,Chemistry
7,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,27.2,%,abnormal,Hematocrit,Blood,Hematology
8,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,8.3,g/dL,abnormal,Hemoglobin,Blood,Hematology
9,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,30.6,%,abnormal,MCHC,Blood,Hematology


# Get an equal amount of patients without sleep apnea for comparison

In [17]:
print(f"Total entries: {len(patients_labs_all):,}")
print(f"Unique patients: {len(patients_labs_all['subject_id'].unique()):,}\n")

apena_count = len(patients_labs_apnea['subject_id'].unique())
print(f"Total entries with apnea: {len(patients_labs_apnea):,}")
print(f"Unique patients with apnea: {apena_count:,}\n")

# Get the same amount of patients without sleep apnea
unique_patients_no_apnea_ids = patients_labs_all[~patients_labs_all['subject_id'].isin(patients_labs_apnea['subject_id'])]['subject_id'].unique()
random_patient_ids = pd.Series(unique_patients_no_apnea_ids).sample(n=apena_count, random_state=42)
patients_without_apnea = patients_labs_all[patients_labs_all['subject_id'].isin(random_patient_ids)].copy() # Copy here seems to take away the slice warning
print(f"Total entries without apnea: {len(patients_without_apnea):,}")
print(f"Unique patients without apnea: {len(patients_without_apnea['subject_id'].unique()):,}\n")

# Add has_apnea column
patients_labs_apnea['has_apnea'] = 1
patients_without_apnea['has_apnea'] = 0

# Combine the two datasets
combined_patients = pd.concat([patients_labs_apnea, patients_without_apnea])
print(f"Total entries: {len(combined_patients):,}")
print(f"Unique patients: {len(combined_patients['subject_id'].unique()):,}")

Total entries: 32,011,486
Unique patients: 14,200

Total entries with apnea: 5,370,629
Unique patients with apnea: 398

Total entries without apnea: 1,005,906
Unique patients without apnea: 398

Total entries: 6,376,535
Unique patients: 796


# Keep at most 500 entries for each subject (prevent file from being too large)

In [28]:
# Keep at most 500 entries for each subject (else file is >55GB after encoding)
patients_final = combined_patients.groupby('subject_id').head(500)
print(f"count: {len(patients_final):,}")
print(f"Unique patients: {len(patients_final['subject_id'].unique()):,}")

count: 299,911
Unique patients: 796


# Save Data

In [29]:
# Save file
patients_final.to_csv('patient_data_apnea_mix.csv', index=False, header=True)
patients_final[:10]

,subject_id,gender,age,admission_type,insurance,language,religion,marital_status,ethnicity,diagnosis,value,valueuom,flag,label,fluid,category,has_apnea
0,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,8.5,g/dL,abnormal,Hemoglobin,Blood,Hematology,1
1,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,10.4,%,abnormal,Lymphocytes,Blood,Hematology,1
2,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,29.9,%,abnormal,MCHC,Blood,Hematology,1
3,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,84.5,%,abnormal,Neutrophils,Blood,Hematology,1
4,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,17.0,%,abnormal,RDW,Blood,Hematology,1
5,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,2.96,m/uL,abnormal,Red Blood Cells,Blood,Hematology,1
6,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,38,mEq/L,abnormal,Bicarbonate,Blood,Chemistry,1
7,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,27.2,%,abnormal,Hematocrit,Blood,Hematology,1
8,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,8.3,g/dL,abnormal,Hemoglobin,Blood,Hematology,1
9,735,F,60.0,ELECTIVE,Private,ENGL,CATHOLIC,MARRIED,white,SPINAL METASTASIS/SDA,30.6,%,abnormal,MCHC,Blood,Hematology,1
